In [ ]:
try: import clmm
except:
    import notebook_install
    notebook_install.install_clmm_pipeline(upgrade=False)
    import clmm
import matplotlib.pyplot as plt
import numpy as np
from astropy import units
plt.rcParams['font.family']=['gothambook','gotham','gotham-book','serif']

clmm.__version__


In [ ]:
import clmm.dataops as da
import clmm.galaxycluster as gc
import clmm.theory as theory
from clmm import Cosmology


In [ ]:
from clmm.support import mock_data as mock


In [ ]:
np.random.seed(11)


In [ ]:
cosmo = Cosmology(H0 = 70.0, Omega_dm0 = 0.27 - 0.045, Omega_b0 = 0.045, Omega_k0 = 0.0)


In [ ]:
import random
#cluster_m = random.randint(10,100) # Masse grenzen für den Cluster
#logm = np.log(cluster_m)/np.log(10)
#concentration = random.randint(1,100)
#cluster_ra = 0.
#cluster_dec = 0.
#cluster_z = random.uniform(0.1,0.6)
#ngals = random.randint(50,100000) 
#cluster_1 = cluster_m, concentration, cluster_ra, cluster_dec, cluster_z, ngals
#print(cluster_m)
#print(concentration)
#print(cluster_z)
#print(ngals)
#print(cluster_1)

In [ ]:
import random
random.seed(22) #gibt einem immer die gleichen zufälligen zahlen
import numpy as np
np.set_printoptions(precision=3)   
num = 5
cluster_data = np.zeros((num,6))
for k in range(num):
    cluster_data[k][0] = np.random.randint(10,100) #masse
    cluster_data[k][1] = np.random.randint(1,100)
    cluster_data[k][2] = 0
    cluster_data[k][3] = 0
    cluster_data[k][4] = np.random.uniform(0.1,0.6)
    cluster_data[k][5] = np.random.randint(50,10000)
print(cluster_data)



In [ ]:
import astropy
import numpy as np
ideal_data = []
noisy_data = []

for k in range(num):

    cluster_m = cluster_data[k][0]
    cluster_z = cluster_data[k][4]
    concentration = cluster_data[k][1]
    ngals = int(cluster_data[k][5])
    buggy_data = mock.generate_galaxy_catalog(cluster_m, cluster_z, concentration, cosmo, 'chang13', zsrc_min = cluster_z + 0.1,shapenoise=0.05, photoz_sigma_unscaled=0.05, ngals=ngals) 
    
    #sortiert nachher die galaxien raus die den falschen redshift besitzen
    mask = buggy_data['z'] < cluster_z #die galaxien sollten entfernt werden nicht maskiert...nur vorrübergehend
    #print(buggy_data['z'][mask])
    #print('cluster rotversciebung')
    #print(cluster_data[k][4])
   
    buggy_data['z'] = np.where(buggy_data['z'] < cluster_z, np.random.uniform(cluster_z,cluster_z+0.1), buggy_data['z'])
    #print(buggy_data['z'])
    #print('hats geklappt?')
    mask = buggy_data['z'] < cluster_z
    #print(buggy_data['z'][mask])
    realredshift = buggy_data['z']
    #print(sum(buggy_data['z']<=0))

    ideal_data.append( mock.generate_galaxy_catalog(cluster_m , cluster_z, concentration, cosmo, 'chang13', zsrc_min = cluster_z + 0.1, ngals = ngals)) 
    noisy_data.append(buggy_data)

                                            

#print(ideal_data)
    

In [ ]:
#https://docs.astropy.org/en/stable/table/operations.html#binning
#groupbi
#https://docs.astropy.org/en/stable/table/masking.html


In [ ]:
for k in range(num):
    plt.figure(figsize=(7,5))
    plt.hist(ideal_data[k]['z'], density = True, alpha=0.5, bins = 50, label='ideal data')
    plt.hist(noisy_data[k]['z'], alpha=0.5, density = True, bins = 50, label='noisy data')
    plt.axvline(x = cluster_data[k][4], color='orange', label = 'cluster redshift')
    plt.xlabel(r'$z_{src}$', fontsize = 20)
    plt.ylabel(r'$N(z$)', fontsize = 20)
    plt.legend()
    plt.xlim(0,5)
    

In [ ]:
gc_object =[]

for k in range(num):
    cluster_ra = cluster_data[k][2]
    cluster_dec = cluster_data[k][3]
    cluster_m = cluster_data[k][0]
    cluster_z = cluster_data[k][4]
    concentration = cluster_data[k][1]
    ngals = int(cluster_data[k][5])
    cluster_id = "CL_ideal"
    gc_object.append( clmm.GalaxyCluster(cluster_id, cluster_ra, cluster_dec,
                                  cluster_z, ideal_data[k]))
    gc_object[k].save('ideal_GC_'+str(k)+'.pkl')

gc_object =[]

for k in range(num):
    cluster_ra = cluster_data[k][2]
    cluster_dec = cluster_data[k][3]
    cluster_m = cluster_data[k][0]
    cluster_z = cluster_data[k][4]
    concentration = cluster_data[k][1]
    ngals = int(cluster_data[k][5])
    cluster_id = "CL_noisy"
    gc_object.append( clmm.GalaxyCluster(cluster_id, cluster_ra, cluster_dec,
                                  cluster_z, noisy_data[k]))
    gc_object[k].save('noisy_GC_'+str(k)+'.pkl')


In [ ]:
cl_ideal = []
cl_noisy = []
for k in range(num):
    cl_ideal.append (clmm.GalaxyCluster.load('ideal_GC_'+str(k)+'.pkl'))
    cl_noisy.append (clmm.GalaxyCluster.load('noisy_GC_'+str(k)+'.pkl'))
    

In [ ]:
#for k in range(num):
    #plt.figure(figsize=(7,5))
    #plt.hist(cl_ideal[k].galcat['z'], density = True, bins = 50)
    #plt.axvline(x = cluster_z, color='orange', label = 'cluster redshift')
    #plt.xlabel(r'$z_{src}$', fontsize = 20)
    #plt.ylabel(r'$N(z$)', fontsize = 20)
    #plt.legend()
    #plt.xlim(0,5)
    

In [ ]:
for k in range(num):
    cl_ideal[k].compute_tangential_and_cross_components(geometry="flat")
    cl_noisy[k].compute_tangential_and_cross_components(geometry="flat")
pass


In [ ]:
bin_edges = da.make_bins(0.2, 4, 15, method='evenlog10width')


In [ ]:
help(mask)

In [ ]:

for k in range(num):
    cluster_z = cluster_data[k][4]
    a = cl_ideal[k].make_radial_profile("Mpc",include_empty_bins=True, bins=bin_edges,return_binnumber=True, cosmo=cosmo, gal_ids_in_bins=True,)#return_binnumber=True does not work
    b = cl_noisy[k].make_radial_profile("Mpc",include_empty_bins=True, bins=bin_edges,cosmo=cosmo, gal_ids_in_bins=True,)
    print(np.shape(cl_ideal[k].profile['z']))
    #print(cl_ideal[k].profile['z'])
    maski = cl_ideal[k].profile['z'] > cluster_z
    maskn = cl_noisy[k].profile['z'] > cluster_z
    mask= maski*maskn
    #print(cl_ideal[k].profile['z'])
    #where_z_broken = np.where(cl_ideal[k].profile['z'] < cluster_z)
    cl_ideal[k].profile =  cl_ideal[k].profile[mask]
    cl_noisy[k].profile =  cl_noisy[k].profile[mask]
    print(np.shape(cl_noisy[k].profile['z']))
    #print(cl_ideal[k].profile['z'])
    #print(k)
    #print(a)
pass


In [ ]:
def model_reduced_tangential_shear_singlez(r, logm, z_src, concentration, cluster_z):
    m = 10.**logm
    gt_model = clmm.compute_reduced_tangential_shear(r,
                                                     m, concentration,
                                                     cluster_z, z_src, cosmo,
                                                     delta_mdef=200,
                                                     halo_profile_model='nfw')    
    return gt_model

In [ ]:
def model_reduced_tangential_shear_zdistrib(radius, logm, data, catalog, profile, concentration, cluster_z): 
    m = 10**logm
    gt_model = []
    for i in range(len(radius)):
        
        r = profile['radius'][i]
        galist = profile['gal_id'][i]
        
        z_list = catalog.galcat['z'][galist]
        shear = clmm.compute_reduced_tangential_shear(r, m, concentration, 
                                                      cluster_z, z_list, cosmo, delta_mdef=200, 
                                                      halo_profile_model='nfw')
        if len(galist) == 0:
            gt_model.append(1e-16)
            
        else:
            gt_model.append(np.mean(shear))
            print(np.mean(shear))

    return gt_model

In [ ]:
gt_model_ideal_singlez = []
gt_model_ideal_zdistrib = []
gt_model_noisy_singlez = []
gt_model_noisy_zdistrib = []
r=[]
for k in range(num):
    print(k)
    cluster_ra = cluster_data[k][2]
    cluster_dec = cluster_data[k][3]
    cluster_m = cluster_data[k][0]
    cluster_z = cluster_data[k][4]
    concentration = cluster_data[k][1]
    ngals = int(cluster_data[k][5])
    logm = np.log10(cluster_m)
    r.append(cl_ideal[k].profile['radius'])
    #mask = r < 1.e-11
    #r = np.where(r < 1.e-11, np.random.uniform(1.e-11,3), r)
    #print(cl_ideal[k].profile['z'])
    #print(cl_noisy[k].profile['z'])
    #print(np.shape(cl_noisy[k].profile))
    gt_model_ideal_singlez.append(model_reduced_tangential_shear_singlez(r[k], logm,cl_ideal[k].profile['z'],concentration, cluster_z))
    gt_model_ideal_zdistrib.append(model_reduced_tangential_shear_zdistrib(r[k],logm, ideal_data[k], cl_ideal[k], cl_ideal[k].profile,concentration,cluster_z))
    gt_model_noisy_singlez.append(model_reduced_tangential_shear_singlez(r[k],logm,cl_noisy[k].profile['z'],concentration, cluster_z ))
    gt_model_noisy_zdistrib.append(model_reduced_tangential_shear_zdistrib(r[k],logm,noisy_data[k], cl_noisy[k], cl_noisy[k].profile,concentration, cluster_z ))
    

In [ ]:
for k in range (num):
    print(k)
    cluster_ra = cluster_data[k][2]
    cluster_dec = cluster_data[k][3]
    cluster_m = cluster_data[k][0]
    cluster_z = cluster_data[k][4]
    concentration = cluster_data[k][1]
    ngals = int(cluster_data[k][5])
    logm = np.log10(cluster_m)
    plt.figure(figsize=(20,8))

    plt.subplot(1,2,1)

    plt.title('ideal data', fontsize=20)
    plt.errorbar(r[k],cl_ideal[k].profile['gt'],cl_ideal[k].profile['gt_err'],c='k',linestyle='', 
                 marker='o', label=r'ideal data, $M_{input}$ = %.2e Msun' % cluster_m)
    plt.loglog(r[k],gt_model_ideal_zdistrib[k],'b',  label=r'model w/ zdistrib, $M_{input}$ = %.2e Msun' % cluster_m)
    plt.loglog(r[k],gt_model_ideal_singlez[k],'-y',  label=r'model w/o zdistrib, $M_{input}$ = %.2e Msun' % cluster_m)
    plt.xlabel('r [Mpc]', fontsize = 20)
    plt.ylabel(r'$g_t$', fontsize = 20)
    plt.xlim(min(cl_ideal[k].profile['radius']), max(cl_ideal[k].profile['radius']))
    plt.legend(fontsize = 15)

    plt.subplot(1,2,2)

    plt.title('noisy data', fontsize=20)
    plt.errorbar(r[k],cl_noisy[k].profile['gt'],cl_noisy[k].profile['gt_err'],c='k',linestyle='', 
                 marker='o',label=r'noisy data, $M_{input}$ = %.2e Msun' % cluster_m)
    plt.loglog(r[k],gt_model_noisy_zdistrib[k],'-b', label=r'model w/ zdistrib, $M_{input}$ = %.2e Msun' % cluster_m)
    plt.loglog(r[k],gt_model_noisy_singlez[k],'-y', label=r'model w/o zdistrib, $M_{input}$ = %.2e Msun' % cluster_m)
    plt.xlabel('r [Mpc]', fontsize = 20)
    plt.ylabel(r'$g_t$', fontsize = 20)
    plt.xlim(min(cl_noisy[k].profile['radius']), max(cl_noisy[k].profile['radius']))
    plt.ylim(min(gt_model_ideal_zdistrib[k]),max(gt_model_ideal_zdistrib[k]))
    plt.legend(fontsize = 15)